In [1]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


In [2]:
dblp = mg.read_csv('mur_dblp.csv', key='id')
acm = mg.read_csv('ACM_cleaned.csv', key='id')

In [3]:
ab = mg.AttrEquivalenceBlocker()
c = ab.block_tables(dblp, acm, 'year', 'year', ['title', 'authors', 'year', 'venue'],
                   ['title', 'authors', 'year', 'venue'])

In [4]:
gold = mg.read_csv('dblp-acm_gold.csv', key='pairId')
gold.columns = ['id', 'ltable.id', 'rtable.id', 'label']
g = mg.create_mtable(gold, key='id', ltable=dblp, rtable=acm, 
                     foreign_key_ltable='ltable.id', 
                     foreign_key_rtable='rtable.id')
g.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [5]:
mg.view(g)

In [5]:
ob = mg.OverlapBlocker()
d = ob.block_tables(dblp, acm, 'title', 'title', l_output_attrs=['title', 'authors'], 
                    r_output_attrs=['title', 'authors'],
                    overlap_size=3)

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 6.798 sec


In [6]:
e = ob.block_candset(d, 'authors', 'authors', overlap_size=1)

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 3.328 sec


In [7]:
len(e)

6652

In [8]:
f = ob.block_tables(dblp, acm, 'authors', 'authors', l_output_attrs=['title', 'authors'], 
                    r_output_attrs=['title', 'authors'],
                    overlap_size=2)

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 5.597 sec


In [13]:
z = mg.combine_block_outputs_via_union([e, f, g1])

In [14]:
z.to_csv('tableC.csv')

True

In [15]:
len(f)

59650

In [11]:
f.head()

,_id,ltable.id,rtable.id,ltable.title,ltable.authors,rtable.title,rtable.authors
0,0,conf/sigmod/AbadiC02,191915,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Shoring up persistent applications,"Michael J. Carey, David J. DeWitt, Michael J. ..."
1,1,conf/sigmod/AbadiC02,191931,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",DEC data distributor: for data replication and...,Daniel J. Dietterich
2,2,conf/sigmod/AbadiC02,233356,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Rule languages and internal algebras for rule-...,"Mitch Cherniack, Stanley B. Zdonik"
3,3,conf/sigmod/AbadiC02,276311,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Changing the rules: transformations for rule-b...,"Mitch Cherniack, Stan Zdonik"
4,4,conf/sigmod/AbadiC02,564770,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack"


In [12]:
s = mg.sample_table(c, 50)
s.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)


In [13]:
se = mg.sample_table(e, 120)
se.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [14]:
sf = mg.sample_table(f, 50)
sf.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [12]:
g1 = mg.sample_table(g, 200)


In [17]:
s1 = set(s.index.values)
s2 = set(se.index.values)
s3 = set(sf.index.values)
s4 = set(g1.index.values)

In [18]:
s5 = s1.union(s2).union(s3)

In [19]:
s5.intersection(s4)

{('conf/sigmod/ChanI98', 276336),
 ('conf/sigmod/GeorgeH97', 253362),
 ('conf/sigmod/NgLHM99', 304575),
 ('journals/tods/BaralisW00', 363954),
 ('journals/vldb/ChakrabartiGRS01', 767147)}

In [20]:
g1.reset_index(inplace=True, drop=True)
s.reset_index(inplace=True, drop=True)
se.reset_index(inplace=True, drop=True)

sf.reset_index(inplace=True, drop=True)




In [21]:
l = mg.combine_block_outputs_via_union([g1, s, se, sf])

In [22]:
len(l)

415

In [23]:
mg.view(l)

In [24]:
l['gold'] = 0

In [22]:
g1.head()

,id,ltable.id,rtable.id,label
0,3,conf/sigmod/RinfretOO01,375669,1
1,12,conf/sigmod/HanPLJ01,375786,1
2,36,conf/sigmod/MistryRSR01,375703,1
3,37,conf/sigmod/Larson01,375792,1
4,57,conf/sigmod/AgrawalCN01,375769,1


In [25]:
l['gold'] = 0

In [31]:
l.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [26]:
g1.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [32]:
g_ind = g.index.values
l_ind = l.index.values

In [34]:
lg_ind = set(g_ind).intersection(l_ind)

In [35]:
len(lg_ind)

231

In [27]:
m = set(g1.index.values)
n = set(l.index.values)

In [28]:
l.head()

,,_id,ltable.id,rtable.id,ltable.authors,ltable.title,ltable.venue,ltable.year,rtable.authors,rtable.title,rtable.venue,rtable.year,gold
ltable.id,rtable.id,,,,,,,,,,,,
conf/sigmod/AboulnagaC99,304198,0,conf/sigmod/AboulnagaC99,304198,"Surajit Chaudhuri, Ashraf Aboulnaga",Self-tuning Histograms: Building Histograms Wi...,SIGMOD Conference,1999,"Ashraf Aboulnaga, Surajit Chaudhuri",Self-tuning histograms: building histograms wi...,International Conference on Management of Data,1999,0
conf/sigmod/AcharyaFZ97,253293,1,conf/sigmod/AcharyaFZ97,253293,"Swarup Acharya, Stanley B. Zdonik, Michael J. ...",Balancing Push and Pull for Data Broadcast,SIGMOD Conference,1997,"Swarup Acharya, Michael Franklin, Stanley Zdonik",Balancing push and pull for data broadcast,International Conference on Management of Data,1997,0
conf/sigmod/AcharyaGP00,335476,2,conf/sigmod/AcharyaGP00,335476,"Viswanath Poosala, Swarup Acharya, Phillip B. ...",Congressional Samples for Approximate Answerin...,SIGMOD Conference,2000,"Jennifer Widom, Adam Bosworth, Bruce Lindsay, ...",Of XML and databases (panel session): where's ...,International Conference on Management of Data,2000,0
conf/sigmod/AcharyaGPR99a,304207,3,conf/sigmod/AcharyaGPR99a,304207,"Swarup Acharya, Viswanath Poosala, Phillip B. ...",Join Synopses for Approximate Query Answering,SIGMOD Conference,1999,"Swarup Acharya, Phillip B. Gibbons, Viswanath ...",Join synopses for approximate query answering,International Conference on Management of Data,1999,0
conf/sigmod/AdaliCPS96,369277,4,conf/sigmod/AdaliCPS96,369277,"Sibel Adali, Yannis Papakonstantinou, K. Seluk...",Query Caching and Optimization in Distributed ...,SIGMOD Conference,1996,"W. Hasselbring, W.-J. van den Heuvel, G. J. Ho...",Research and practice in federated information...,ACM SIGMOD Record,2000,0


In [36]:
l.ix[lg_ind, 'gold'] = 1

In [37]:
mg.edit(l)

In [38]:
len(m.intersection(n))

NameError: name 'm' is not defined

In [39]:
l.reset_index(drop=True, inplace=True)

In [40]:
mg.view(l)

In [41]:
l.to_csv('acm_dblp_label_4.csv')

True